In [1]:
!pip install tensorflow-gpu


     |████████████████████████████████| 320.4MB 51kB/s 


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Dataset/train.csv")
store_df = pd.read_csv("/content/drive/MyDrive/Dataset/store.csv")
df = pd.merge(df, store_df, how='inner', on='Store')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   StoreType                  1017209 non-null  object 
 10  Assortment                 1017209 non-null  object 
 11  CompetitionDistance        1014567 non-null  float64
 12  CompetitionOpenSinceMonth  693861 non-null   float64
 13  CompetitionO

In [6]:
#Data Preprocessing
df = df.drop(df[df.Open == 0].index)
df = df.drop(df[df.Sales == 0].index)

In [7]:
# Convert for time series
df['Date'] = df['Date'].astype('datetime64[ns]')

In [8]:
store_df.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [9]:
df['CompetitionDistance'].fillna(store_df['CompetitionDistance'].median(), inplace = True)

In [10]:
df.fillna(0, inplace = True)

In [11]:
df['StateHoliday'] = df.StateHoliday.replace([0, '0'], np.nan)
df.StateHoliday.unique()

array([nan, 'a', 'b', 'c'], dtype=object)

In [12]:
dummy_columns = ['StoreType', 'Assortment', 'StateHoliday']
df = pd.get_dummies(df, columns=dummy_columns)

In [13]:
#Splitting the Date as Day, month and year and adding 3 new columns
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Weekofyear'] = df['Date'].dt.weekofyear

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


In [14]:
#How long the competition has been open
df['CompetitionOpen'] = 12 * (df.Year - df.CompetitionOpenSinceYear) + (df.Month - df.CompetitionOpenSinceMonth)

In [15]:
#How long the promo has been running
df['PromoOpen'] = 12 * (df.Year - df.Promo2SinceYear) + (df.Weekofyear - df.Promo2SinceWeek) / 4.0

In [16]:
df = df.drop(columns=['Weekofyear','Promo2SinceWeek', 'CompetitionOpenSinceMonth', 'Promo2SinceYear', 'CompetitionOpenSinceYear', 'Open', 'PromoInterval'])

In [17]:
#replace competitiondistance with furthest variable
df['CompetitionDistance'] = df.CompetitionDistance.replace(np.nan, df['CompetitionDistance'].max())
                                                           
#Competittion Open - use mean
df['CompetitionOpen'] = df.CompetitionOpen.replace(np.nan, df['CompetitionOpen'].max())

#PromoOpen - recode missing as 0
df['PromoOpen'] = df.PromoOpen.replace(np.nan, 0)

In [18]:
from scipy.stats import zscore

z_scores = np.abs(zscore(df.Sales))
z_scores

array([0.54544483, 0.62373562, 0.70041548, ..., 0.77838407, 0.85667486,
       1.04998543])

In [19]:
removed_df = df[z_scores < 3]
print('Removed data', len(df) - len(removed_df))
df = removed_df

Removed data 13443


In [20]:
X = df.drop(columns=['Date','Sales','Customers','Store','StateHoliday_b','StateHoliday_c'])
Y = df['Sales']

In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)
scaled_X

array([[ 0.85442228,  1.12330986,  2.04438985, ...,  1.50221827,
        -0.68066285,  1.00770216],
       [ 0.27291095,  1.12330986,  2.04438985, ...,  1.50221827,
        -0.68066285,  1.00770216],
       [-0.30860038,  1.12330986,  2.04438985, ...,  1.50221827,
        -0.68066285,  1.00770216],
       ...,
       [ 0.85442228, -0.89022632,  2.04438985, ..., -1.07021472,
         1.46364546, -0.99514049],
       [ 0.27291095, -0.89022632,  2.04438985, ..., -1.07021472,
         1.46364546, -0.99514049],
       [-0.30860038, -0.89022632,  2.04438985, ..., -1.07021472,
         1.46364546, -0.99514049]])

In [22]:
from sklearn.model_selection import train_test_split

x_train, x_rest, y_train, y_rest = train_test_split(scaled_X, Y, test_size=0.30)
x_test, x_val, y_test, y_val = train_test_split(x_rest, y_rest, test_size=0.5)
print(len(x_train) + len(x_test) + len(x_val) == len(df))
print(x_train)
print(x_test)
x_val

True
[[ 1.43593361 -0.89022632 -0.4891435  ... -1.07021472 -0.6819098
  -0.99632078]
 [ 0.27291095  1.12330986 -0.4891435  ...  0.21600177 -0.6788815
  -0.99261427]
 [ 0.85442228 -0.89022632 -0.4891435  ... -1.07021472  1.4640908
  -0.99346325]
 ...
 [ 0.85442228 -0.89022632 -0.4891435  ... -1.07021472  1.4640908
  -0.99526474]
 [ 1.43593361 -0.89022632 -0.4891435  ...  0.21600177  1.46560495
  -0.99275921]
 [-0.30860038 -0.89022632 -0.4891435  ... -1.07021472  1.4640908
  -0.99572028]]
[[ 0.85442228 -0.89022632 -0.4891435  ... -1.07021472 -0.68208794
   1.00525876]
 [-0.30860038  1.12330986 -0.4891435  ...  1.50221827  1.46622842
   1.00757792]
 [-1.47162305  1.12330986 -0.4891435  ...  0.21600177 -0.68858989
   1.00685318]
 ...
 [-1.47162305 -0.89022632 -0.4891435  ... -1.07021472  1.46364546
  -0.99563746]
 [-1.47162305 -0.89022632 -0.4891435  ... -1.07021472 -0.6863632
  -0.99489201]
 [-0.30860038 -0.89022632  2.04438985 ... -1.07021472  1.46417986
  -0.99509908]]


array([[-0.30860038,  1.12330986, -0.4891435 , ...,  0.21600177,
        -0.68262235, -0.99489201],
       [ 0.27291095,  1.12330986, -0.4891435 , ...,  0.21600177,
        -0.6888571 ,  1.00654258],
       [-1.47162305,  1.12330986, -0.4891435 , ..., -1.07021472,
        -0.68582879, -0.99725259],
       ...,
       [-0.30860038,  1.12330986,  2.04438985, ...,  0.21600177,
         1.46542681,  1.00681177],
       [ 0.27291095, -0.89022632,  2.04438985, ...,  1.50221827,
        -0.6741609 , -0.99242791],
       [-0.89011171,  1.12330986, -0.4891435 , ...,  0.21600177,
         1.46542681,  1.0068946 ]])

In [23]:
x_train.shape

(581626, 18)

In [24]:
x_test.shape

(124634, 18)

In [25]:
#Since the input for a CNN and LSTM should be in 3 dimension, the inputs were reshaped.
# reshape input to be [samples, time steps, features] which is required for LSTM
x_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0], 1, x_test.shape[1])

In [26]:
x_train.shape

(581626, 1, 18)

In [27]:
x_test.shape

(124634, 1, 18)

In [28]:
#Modelling Part

import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import LSTM
from keras.utils import np_utils

In [41]:
model = Sequential()
model.add(LSTM(16,return_sequences=True,input_shape=(1, 18), activation = 'tanh', kernel_initializer = 'glorot_uniform',
              kernel_regularizer = 'l2'))
model.add(Dropout(0.2))
model.add(LSTM(8,return_sequences=True, activation = 'tanh'))
model.add(Dropout(0.2))
model.add(LSTM(8, activation = 'linear'))
model.add(Dropout(0.2))
model.add(Dense(1))

In [42]:
model.compile(loss='mean_squared_error',optimizer='adam', metrics = ['mse'])

In [43]:
lstm_model = model.fit(x_train, y_train, epochs=200, verbose=1)

Epoch 1/200
18176/18176 [==============================] - 63s 3ms/step - loss: 19106118.0000 - mse: 19106118.0000
Epoch 2/200
18176/18176 [==============================] - 65s 4ms/step - loss: 7784532.0000 - mse: 7784530.0000
Epoch 3/200
18176/18176 [==============================] - 67s 4ms/step - loss: 7406446.0000 - mse: 7406444.0000
Epoch 4/200
18176/18176 [==============================] - 66s 4ms/step - loss: 7145184.0000 - mse: 7145181.0000
Epoch 5/200
18176/18176 [==============================] - 66s 4ms/step - loss: 6937489.0000 - mse: 6937484.5000
Epoch 6/200
18176/18176 [==============================] - 63s 3ms/step - loss: 6784881.0000 - mse: 6784877.0000
Epoch 7/200
18176/18176 [==============================] - 61s 3ms/step - loss: 6669318.5000 - mse: 6669315.0000
Epoch 8/200
18176/18176 [==============================] - 61s 3ms/step - loss: 6595781.0000 - mse: 6595776.5000
Epoch 9/200
18176/18176 [==============================] - 61s 3ms/step - loss: 6553480.5000 -

In [44]:
predict = model.predict(x_test)

In [45]:
from sklearn import metrics

In [46]:
# Print the results of MAE
print(np.sqrt(metrics.mean_absolute_error(y_test, predict)))

# Print the results of MSE
print(metrics.mean_squared_error(y_test, predict))

# Print the results of RMSE
print(np.sqrt(metrics.mean_squared_error(y_test, predict)))

40.21948486865622
4483046.168513649
2117.320516245391
